## First a UKF implemented on a nonlinear pendulum system

In [1]:
'''
Simulating the equation $mL^2\ddot{\theta} + b \dot{\theta} +mgL sin(\theta) = 0$
'''
import numpy as np
import cv2
import time
# UKF library
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints

# visulaization parameters
height = 600
width = 600

# simulation 
center = None

# Pendulum parameters and variables

#    initial conditions
theta = np.pi/2.*.5
omega = 0

#    parametrs
g = 980 #9.8 # m/s^2
L = 240 # m
m = 0.05 # kg
b = 0 


# Numerical integration parameters
framerate = 60.0 # in frames per second
dt = 1.0/framerate # Set dt to match the framerate of the webcam or animation
t = time.clock()

# Drawing parametres
thickness = 3

# Noise parameters 
# Sigma = 0.002 * np.array([[1, 0],[0,1]])
covariance_between_states = 1
measurement_noise_std = 25
# process_noise = 10
'''
# Kalman inferred state variables (g-h model like right now)
theta_kf = theta
omega_kf = omega
theta_kf_old = theta_kf'''
# Keep looping

# Create background image
frame = np.zeros((height,width,3), np.uint8)

map_cerner_old = (300, 300)
map_cerner_noisy_old = (300, 300)
center_kf_old = (300, 300)

# Create background image
frame = np.zeros((height,width,3), np.uint8)

cv2.circle(frame, (300, 300), 10, (0, 255, 255), -1)

# define the state traL^2\ddot{\theta} + b \dot{\theta} +mgL sin(\theta) = 0nsition function (not a matrix anymore) - predict next state 
# state is [theta, theta_dot]
def f_cv(x, dt):
    theta_ = x[0] + dt * x[1]
    theta_dot_ = x[1] - dt*g/L*np.sin(theta_) #  theta_dot = theta_dot - dt*g/L*np.sin(theta)
    # L_ = np.sqrt(np.power((np.sin(x[0])*x[2]),2)+np.power((np.cos(x[0])*x[2]),2))
    return [theta_, theta_dot_]

# measurement function - convert state into a measurement where measurements are [x, y]
def h_cv(x):
    theta_ = x[0]
    pos_x = L*np.sin(theta_)
    pos_y = L*np.cos(theta_)
    return [pos_x, pos_y]

# == UKF Model ==
points = MerweScaledSigmaPoints(2, alpha=1e-3, beta=2., kappa=1.0)
# dim_x=2: state is 2: theta, theta_dot; dim_z=: theta
ukf = UKF(dim_x=2, dim_z=2, fx=f_cv, hx=h_cv, dt=dt, points=points) 

ukf.P = np.diag([covariance_between_states, covariance_between_states]) # covariance between states      
ukf.x = np.array([0., 0.]) # initial state
ukf.R = np.diag([measurement_noise_std**2, measurement_noise_std**2]) # measurement noise matrix (dim_z, dim_z)
# ukf.Q = np.diag([process_noise, process_noise]) # process noise matrix (dim_x, dim_x)
ukf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=10, block_size=1)

while True:
    cv2.circle(frame, (300, 300), 10, (0, 255, 255), -1)

    # update the state
    theta = theta + dt*omega
    omega = omega - dt*g/L*np.sin(theta) - dt*b/(m*L*L)*omega
    # calculate the center
    center = np.array((L*np.sin(theta) ,L*np.cos(theta))) 
#     print('center', center)
    ################################# measurement_noise_std
    center_noisy = center + (measurement_noise_std * np.random.randn(2))
#     print('center_noisy', center_noisy)
    # map the center to the nearest pixel
    # map_center is the coordinates in the opencv window
    map_cerner = (int(300 + center[0]), int(300 + center[1]))
    map_cerner_noisy = (int(300 + center_noisy[0]), int(300 + center_noisy[1]))	
    # Draw the pendulum
    cv2.circle(frame, map_cerner_old, 10, (0, 0, 0), -1)
    cv2.circle(frame, map_cerner_noisy_old, 10, (0, 0, 0), -1)
    cv2.circle(frame, map_cerner, 10, (0, 255, 255), -1)
    cv2.circle(frame, map_cerner_noisy, 10, (0, 0, 255), -1)
    map_cerner_old = map_cerner
    map_cerner_noisy_old = map_cerner_noisy
    
    ##################   UKF  #################################
    # UKF predict and update
#     print('real state', theta, omega)
    # predict step
    ukf.predict()
#     print('after predict', ukf.x[0], ukf.x[1])
    
    # update step using the measurements
    ukf.update([center_noisy[0], center_noisy[1]])
#     print('after update', ukf.x[0], ukf.x[1])
#     print('kalman gain', ukf.K) 
#     print('  ')

    # map the UKF ouput to the nearest pixel
    center_kf = np.array((int(300 + np.sin(ukf.x[0])*L), int(300 + np.cos(ukf.x[0])*L))) 
    cv2.circle(frame, tuple(center_kf_old), 10, (0, 0, 0), -1)
    center_kf_old = center_kf
    cv2.circle(frame, tuple(center_kf), 10, (255, 0, 255), -1)
    
    # show the frame to our screen
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(int(dt*400)) & 0xFF

    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
    
    
    # Wait with calculating next animation step to match the intended framerate
    t_ready = time.clock()
    d_t_animation = t + dt -  t_ready
    t += dt
    if  d_t_animation > 0:
        time.sleep(d_t_animation)


# close all windows
cv2.destroyAllWindows()

/home/mt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/mt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:143: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


### Below is a dual estimation of both clean state (theta, theta_dot) and system parameter (length of the pendulum)

In this example, the state is [theta, theta_dot, length]. After the simulation starts, we can see the predicted positon of pendulum is converging to the ture position.

In [6]:
'''
Simulating the equation $mL^2\ddot{\theta} + b \dot{\theta} +mgL sin(\theta) = 0$
'''
import numpy as np
import cv2
import time
# UKF library
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints

# visulaization parameters
height = 600
width = 600

# simulation 
center = None

# Pendulum parameters and variables

#    initial conditions
theta = np.pi/2.*.5
omega = 0

#    parametrs
g = 980 #9.8 # m/s^2
L = 240 # m
m = 0.05 # kg
b = 0 


# Numerical integration parameters
framerate = 60.0 # in frames per second
dt = 1.0/framerate # Set dt to match the framerate of the webcam or animation
t = time.clock()

# Drawing parametres
thickness = 3

# Noise parameters 
covariance_between_states = 1
measurement_noise_std = 25

# Keep looping

# Create background image
frame = np.zeros((height,width,3), np.uint8)

map_cerner_old = (300, 300)
map_cerner_noisy_old = (300, 300)
center_kf_old = (300, 300)

# Create background image
frame = np.zeros((height,width,3), np.uint8)

cv2.circle(frame, (300, 300), 10, (0, 255, 255), -1)

# define the state traL^2\ddot{\theta} + b \dot{\theta} +mgL sin(\theta) = 0nsition function (not a matrix anymore) - predict next state 
# state is [theta, theta_dot, L]
def f_cv(x, dt):
    theta_ = x[0] + dt * x[1]
    theta_dot_ = x[1] - dt*g/L*np.sin(theta_) #  theta_dot = theta_dot - dt*g/L*np.sin(theta)
    L_ = x[2] # np.sqrt(np.power((np.sin(x[0])*x[2]),2)+np.power((np.cos(x[0])*x[2]),2))
    return [theta_, theta_dot_, L_]

# measurement function - convert state into a measurement where measurements are [x, y]
def h_cv(x):
    theta_ = x[0]
    length = x[2]
    pos_x = length*np.sin(theta_)
    pos_y = length*np.cos(theta_)
    return [pos_x, pos_y]

# == UKF Model ==
points = MerweScaledSigmaPoints(3, alpha=1e-3, beta=2., kappa=1.0)
# dim_x=3: state is 3: theta, theta_dot, length; dim_z=2: x and y
ukf = UKF(dim_x=3, dim_z=2, fx=f_cv, hx=h_cv, dt=dt, points=points) 
ukf.P = np.diag([covariance_between_states, covariance_between_states, covariance_between_states]) # covariance between states      
ukf.x = np.array([0., 0., 0.]) # initial state
ukf.R = np.diag([measurement_noise_std**2, measurement_noise_std**2]) # measurement noise matrix (dim_z, dim_z)
# process noise matrix Q
# ukf.Q = np.zeros((3,3))
ukf.Q = Q_discrete_white_noise(dim=3, dt=dt, var=0.1, block_size=1)
# ukf.Q[2,2] = 0.1

while True:
    cv2.circle(frame, (300, 300), 10, (0, 255, 255), -1)

    # update the state
    theta = theta + dt*omega
    omega = omega - dt*g/L*np.sin(theta) - dt*b/(m*L*L)*omega
    # calculate the center
    center = np.array((L*np.sin(theta) ,L*np.cos(theta))) 
#     print('center', center)
    ## measurement_noise_std
    center_noisy = center + (measurement_noise_std * np.random.randn(2))
#     print('center_noisy', center_noisy)
    # map the center to the nearest pixel
    map_cerner = (int(300 + center[0]), int(300 + center[1]))
    map_cerner_noisy = (int(300 + center_noisy[0]), int(300 + center_noisy[1]))	
    # Draw the pendulum
    cv2.circle(frame, map_cerner_old, 10, (0, 0, 0), -1)
    cv2.circle(frame, map_cerner_noisy_old, 10, (0, 0, 0), -1)
    cv2.circle(frame, map_cerner, 10, (0, 255, 255), -1)
    cv2.circle(frame, map_cerner_noisy, 10, (0, 0, 255), -1)
    map_cerner_old = map_cerner
    map_cerner_noisy_old = map_cerner_noisy
    
    ##################   UKF  #################################
    # UKF predict and update
#     print('real state', theta, omega, L)
    # predict step
    ukf.predict()
#     print('after predict', ukf.x[0], ukf.x[1], ukf.x[2])
    
    # update step using the measurements
    ukf.update([center_noisy[0], center_noisy[1]])
#     print('after update', ukf.x[0], ukf.x[1], ukf.x[2])
#     print('kalman gain', ukf.K) 
#     print('  ')

    # map the UKF ouput to the nearest pixel
    center_kf = np.array((int(300 + np.sin(ukf.x[0])*ukf.x[2]), int(300 + np.cos(ukf.x[0])*ukf.x[2]))) 
    cv2.circle(frame, tuple(center_kf_old), 10, (0, 0, 0), -1)
    center_kf_old = center_kf
    cv2.circle(frame, tuple(center_kf), 10, (255, 0, 255), -1)
    
    # show the frame to our screen
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(int(dt*400)) & 0xFF

    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
    
    
    # Wait with calculating next animation step to match the intended framerate
    t_ready = time.clock()
    d_t_animation = t + dt -  t_ready
    t += dt
    if  d_t_animation > 0:
        time.sleep(d_t_animation)


# close all windows
cv2.destroyAllWindows()

/home/mt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/mt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:138: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


As can be seen in the simulation, the length of the pendulum is converging to the true length, and the predict coordinats is also converging to the true location.